# Train and Deploy the Model


In [3]:
%store -r n_user
%store -r n_item

print(n_user)
print(n_item)

39
2418


In [4]:
!pip install --upgrade tensorflow

     |██████████████████████          | 316.2 MB 109.6 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 458.3 MB 10.0 kB/s             
     |████████████████████████████████| 462 kB 105.4 MB/s            
     |████████████████████████████████| 42 kB 2.2 MB/s             
     |████████████████████████████████| 65 kB 927 kB/s              
     |████████████████████████████████| 1.3 MB 97.9 MB/s            
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 132 kB 98.0 MB/s            
     |████████████████████████████████| 5.6 MB 80.1 MB/s            
     |████████████████████████████████| 4.1 MB 56.9 MB/s            
     |████████████████████████████████| 4.9 MB 83.3 MB/s            
     |████████████████████████████████| 152 kB 127.0 MB/s            
     |████████████████████████████████| 781 kB 77.3 MB/s            
     |████████████████████████████████| 97 kB 15.3 MB/s             
     |████████████████████████████████| 155 kB 120.5 MB/s            
     |██████████

In [5]:
import os
import json
import sagemaker
import numpy as np
import pandas as pd
import tensorflow as tf
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

# get current SageMaker session's execution role and default bucket name
sagemaker_session = sagemaker.Session()
role = get_execution_role()
print("execution role ARN:", role)
bucket_name = sagemaker_session.default_bucket()
print("default bucket name:", bucket_name)

execution role ARN: arn:aws:iam::452957727655:role/c40110a511491l1281465t1w452-SageMakerExecutionRole-BMF8O4RGVJ1V
default bucket name: sagemaker-us-east-1-452957727655


In [6]:
# specify the location of the training data
training_data_uri = os.path.join(f's3://{bucket_name}', 'data')

In [7]:
# inspect the training script using `pygmentize` magic
!pygmentize 'ncf1.py'

"""

 Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
 SPDX-License-Identifier: MIT-0
 
 Permission is hereby granted, free of charge, to any person obtaining a copy of this
 software and associated documentation files (the "Software"), to deal in the Software
 without restriction, including without limitation the rights to use, copy, modify,
 merge, publish, distribute, sublicense, and/or sell copies of the Software, and to
 permit persons to whom the Software is furnished to do so.

 THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
 INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
 PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
 HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
 OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
 SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

"""


import tensorflow

In [8]:
# specify training instance type and model hyperparameters
# note that for the demo purpose, the number of epoch is set to 1

num_of_instance = 1                 # number of instance to use for training
instance_type = 'ml.c5.2xlarge'     # type of instance to use for training

training_script = 'ncf1.py'

training_parameters = {
    'epochs': 1,
    'batch_size': 256, 
    'n_user': n_user, 
    'n_item': n_item
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}

In [9]:
# initiate the training job using Tensorflow estimator
ncf_estimator = TensorFlow(
    entry_point=training_script,
    role=role,
    train_instance_count=num_of_instance,
    train_instance_type=instance_type,
    framework_version=tensorflow_version,
    py_version=python_version,
    distributions=distributed_training_spec,
    hyperparameters=training_parameters
)

distributions has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [10]:
# kick off the training job
ncf_estimator.fit(training_data_uri)

2022-02-14 08:20:26 Starting - Starting the training job...
2022-02-14 08:20:49 Starting - Launching requested ML instancesProfilerReport-1644826826: InProgress
.........
2022-02-14 08:22:10 Starting - Preparing the instances for training......
2022-02-14 08:23:21 Downloading - Downloading input data
2022-02-14 08:23:21 Training - Downloading the training image...
2022-02-14 08:23:55 Uploading - Uploading generated training model
2022-02-14 08:23:55 Completed - Training job completed
2022-02-14 08:23:42,206 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2022-02-14 08:23:42,212 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-02-14 08:23:42,675 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-02-14 08:23:42,690 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-02-14 08:23:42,703 sagemaker-containers INFO     No GPUs detected (normal if no gpus insta

In [16]:
!pip install bitmex-websocket
!pip install bitmex-ws

## Deploy the Endpoint

In [29]:
predictor = ncf_estimator.deploy( initial_instance_count=1, 
                                 instance_type='ml.c5.xlarge', 
                                 endpoint_name= 'modeldemo'
                                )

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

## Invoke

In [30]:
from sagemaker.tensorflow import TensorFlowPredictor #initiate a predictor
predictor = TensorFlowPredictor(endpoint_name)

In [31]:
def _load_testing_data(base_dir):
    """ load testing data """
    df_test = np.load(os.path.join(base_dir, 'test.npy'))
    user_test, item_test, y_test = np.split(np.transpose(df_test).flatten(), 3)
    return user_test, item_test, y_test

In [32]:
# read testing data from local
user_test, item_test, test_labels = _load_testing_data('./bollywoodsongs/s3/')

# one-hot encode the testing data for model input
with tf.compat.v1.Session() as tf_sess:
    test_user_data = tf_sess.run(tf.one_hot(user_test, depth=n_user)).tolist()
    test_item_data = tf_sess.run(tf.one_hot(item_test, depth=n_item)).tolist()

In [33]:
# make batch prediction
batch_size = 150
y_pred = []
for idx in range(0, len(test_user_data), batch_size):
    # reformat test samples into tensorflow serving acceptable format
    input_vals = {
     "instances": [
         {'input_1': u, 'input_2': i} 
         for (u, i) in zip(test_user_data[idx:idx+batch_size], test_item_data[idx:idx+batch_size])
    ]}
 
    # invoke model endpoint to make inference
    pred = predictor.predict(input_vals)
    
    # store predictions
    y_pred.extend([i[0] for i in pred['predictions']])

In [43]:
# let's see some prediction examples, assuming the threshold 
# --- prediction probability view ---
print('This is what the prediction output looks like')
print(y_pred[:10], end='\n\n\n')

# --- user item pair prediction view, with threshold of 0.5 applied ---
pred_df = pd.DataFrame([
    user_test,
    item_test,
    (np.array(y_pred) >= 0.4468).astype(int)],
).T

pred_df.columns = ['UserID', 'SongID', 'prediction']

print('We can convert the output to user-item pair as shown below')
print(pred_df.head(8), end='\n\n\n')

# --- aggregated prediction view, by user ---
print('Lastly, we can roll up the prediction list by user and view it that way')
print(pred_df.query('prediction == 1').groupby('UserID').SongID.apply(list).head(8).to_frame(), end='\n\n\n')

This is what the prediction output looks like
[0.447166145, 0.447723567, 0.442998976, 0.443850935, 0.447030425, 0.449374944, 0.446814775, 0.449562, 0.450790644, 0.448371232]


We can convert the output to user-item pair as shown below
   UserID  SongID  prediction
0       1       1           1
1       1      13           1
2       1      12           0
3       1      11           0
4       1       9           1
5       1       8           1
6       1      10           1
7       1       6           1


Lastly, we can roll up the prediction list by user and view it that way
                                                   SongID
UserID                                                   
1                              [1, 13, 9, 8, 10, 6, 5, 4]
14                   [21, 25, 23, 22, 20, 19, 18, 17, 16]
15                           [65, 64, 63, 61, 66, 57, 56]
26               [26, 27, 28, 29, 30, 31, 32, 34, 35, 36]
45              [97, 98, 99, 100, 101, 103, 104, 105, 96]
75      [118, 1

## Delete Endpoint

In [24]:
# delete endpoint at the end of the demo
predictor.delete_endpoint(delete_endpoint_config=True)